In [1]:
pip install --upgrade flair

     |████████████████████████████████| 337kB 3.4MB/s 
     |████████████████████████████████| 71kB 5.8MB/s 
     |████████████████████████████████| 19.7MB 1.2MB/s 
     |████████████████████████████████| 798kB 53.4MB/s 
     |████████████████████████████████| 276kB 49.3MB/s 
     |████████████████████████████████| 983kB 57.7MB/s 
     |████████████████████████████████| 1.3MB 57.2MB/s 
     |████████████████████████████████| 1.1MB 56.2MB/s 
     |████████████████████████████████| 2.9MB 63.2MB/s 
     |████████████████████████████████| 890kB 55.0MB/s 
  Created wheel for segtok: filename=segtok-1.5.10-cp36-none-any.whl size=25021 sha256=e869a1dd0c415a6206d1fe00c2bf4b49e9cdfa078bf3ae94e95a0079b6a61e7d
  Stored in directory: /root/.cache/pip/wheels/b4/39/f6/9ca1c5cabde964d728023b5751c3a206a5c8cc40252321fb6b
  Created wheel for ftfy: filename=ftfy-5.8-cp36-none-any.whl size=45612 sha256=d754f85a6dc72644cd4b9ed4f21b3185e14958a4f7e4f40d611d9b23e3cc7214
  Stored in directory: /root/.cache/pip

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/' ,force_remount=True)



Mounted at /content/gdrive/


In [ ]:
from flair.data import Corpus
import gensim

from flair.embeddings import FastTextEmbeddings,TokenEmbeddings, CharacterEmbeddings,  WordEmbeddings, StackedEmbeddings, PooledFlairEmbeddings,FlairEmbeddings,DocumentPoolEmbeddings
from typing import List
from flair.data import Dictionary
from flair.models import LanguageModel
from flair.trainers.language_model_trainer import LanguageModelTrainer, TextCorpus
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import BertEmbeddings
import re
import numpy as np
from nltk import ngrams

# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = '/content/gdrive/My Drive/resources/tasks/conll_03'


# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='dev.txt')
    
    

# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

embedding_types: List[TokenEmbeddings] = [

    # fastext embeddings
    #WordEmbeddings(data_folder+'/model.bin',True),
    #WordEmbeddings(data_folder+'/full_grams_cbow_300_wiki.mdl'),
    #BertEmbeddings('bert-base-multilingual-cased'),
    #WordEmbeddings( data_folder+'/modelsg300wiki.gensim', 'custom'),
    #CharacterEmbeddings(path_to_char_dict=data_folder+'/pytorch_model.bin',True)
    #BertEmbeddings(bert_model_or_path = data_folder+'/config.json', data_folder+'/pytorch_model.bin' ,  data_folder+'/vocab.txt'), 
    BertEmbeddings(bert_model_or_path = data_folder),
    #flair fast embedding 
    WordEmbeddings('ar'),
  

    # orignal fast embeddings  from face
    #FastTextEmbeddings(data_folder+'/fasttext300.bin'),
    # BytePairEmbeddings('multi'),
    # contextual string embeddings, forward
    PooledFlairEmbeddings('ar-forward',  pooling= 'mean'),
    # contextual string embeddings, backward
    PooledFlairEmbeddings('ar-backward' ,  pooling= 'mean'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

#embeddings = DocumentPoolEmbeddings(embeddings=embedding_types)

# initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type  )

# initialize trainer
from flair.trainers import ModelTrainer
from torch.optim.adam import Adam
trainer: ModelTrainer = ModelTrainer(tagger, corpus)
                                        
                            
import pickle

trainer.train('/content/gdrive/My Drive/resources/taggers/example-ner',
              train_with_dev=True, 
              learning_rate=0.1,   
              mini_batch_size=16,
              max_epochs=150,
              checkpoint=True)



2020-10-22 16:15:24,977 Reading data from /content/gdrive/My Drive/resources/tasks/conll_03
2020-10-22 16:15:24,977 Train: /content/gdrive/My Drive/resources/tasks/conll_03/train.txt
2020-10-22 16:15:24,978 Dev: /content/gdrive/My Drive/resources/tasks/conll_03/dev.txt
2020-10-22 16:15:24,979 Test: /content/gdrive/My Drive/resources/tasks/conll_03/test.txt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: DeprecationWarning: Call to deprecated method __init__. (Use 'TransformerWordEmbeddings' for all transformer-based word embeddings) -- Deprecated since version 0.4.5.


2020-10-22 16:15:39,897 https://flair.informatik.hu-berlin.de/resources/embeddings/token/ar-wiki-fasttext-300d-1M.vectors.npy not found in cache, downloading to /tmp/tmp6_vs5jes


100%|██████████| 733171328/733171328 [00:31<00:00, 23494770.10B/s]

2020-10-22 16:16:11,535 copying /tmp/tmp6_vs5jes to cache at /root/.flair/embeddings/ar-wiki-fasttext-300d-1M.vectors.npy


...

2020-10-22 16:16:34,765 copying /tmp/tmp6ctggpez to cache at /root/.flair/embeddings/lm-ar-opus-large-forward-v0.1.pt


2020-10-22 16:16:34,897 removing temp file /tmp/tmp6ctggpez
2020-10-22 16:16:51,000 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-ar-opus-large-backward-v0.1.pt not found in cache, downloading to /tmp/tmpvrlc3mo4


100%|██████████| 131796811/131796811 [00:06<00:00, 21315076.40B/s]

2020-10-22 16:16:57,712 copying /tmp/tmpvrlc3mo4 to cache at /root/.flair/embeddings/lm-ar-opus-large-backward-v0.1.pt


2020-10-22 16:16:57,836 removing temp file /tmp/tmpvrlc3mo4
2020-10-22 16:17:02,187 ----------------------------------------------------------------------------------------------------
2020-10-22 16:17:02,191 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): BertEmbeddings(
      (model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(64000, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=

In [ ]:
from flair.data import Corpus
from flair.datasets import CONLL_03
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, PooledFlairEmbeddings
from typing import List
from flair.data import Dictionary
from flair.models import LanguageModel
from flair.trainers.language_model_trainer import LanguageModelTrainer, TextCorpus
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = '/content/gdrive/My Drive/resources/tasks/conll_03'
#data_folder = '/content/gdrive/My Drive/conll_03'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='dev.txt')
    


# initialize trainer
from flair.trainers import ModelTrainer

checkpoint = '/content/gdrive/My Drive/resources/taggers/example-ner/checkpoint.pt'
from pathlib import Path
from torch.optim.adam import Adam
trainer = ModelTrainer.load_checkpoint(checkpoint, corpus)

trainer.train('/content/gdrive/My Drive/resources/taggers/example-ner',
              train_with_dev=True, 
              learning_rate=0.1,   
              mini_batch_size=16,
              max_epochs=150,
              checkpoint=True)


In [ ]:

from flair.models import SequenceTagger
model = SequenceTagger.load('/content/gdrive/My Drive/resources/taggers/example-ner/final-model.pt')
from flair.data import Sentence
# create example sentence
sentence = Sentence('إياد ومريم يلعبون فى الحديقة') 

# predict tags and print
model.predict(sentence)
#model.evaluate()
print(sentence.to_tagged_string('ner'))

for token in sentence:
    tag = token.get_tag('ner')
    print(f'"{token}" is tagged as "{tag.value}" with confidence score "{tag.score}"')